Result 1: score 0.08579 using densenet121 and TTA32. TTA32 is very long - 1 hour, training is almost the same time. I don't like the curve shape of training loss - too zigzagy, so trying another model.

Result 2: Score: 0.17189 using resnet50 training 8 epochs (less 2 epochs than above) and TTA4, also removed ShiftScaleRotate, so I don't know the exact reason for the lower score.

Result 3: Score 0.04953 for softmax() and 0.05880 for just dog clip. Should use softmax(). For this try I am back to densent121, reduced training set to 10% from initial 20%, returned back ShiftScaleRotate() augmentation, trained simple learn.fine_tune(12), used TTA32.

In [ ]:
from fastai.vision.all import *
import pandas as pd
%matplotlib inline
set_seed(3865)

In [ ]:
import zipfile
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [ ]:
import os
# some bad images should be deleted from the train set
os.remove('./train/dog.10797.jpg')
os.remove('./train/dog.10747.jpg')
os.remove('./train/dog.10237.jpg')
os.remove('./train/dog.9517.jpg')
os.remove('./train/dog.8736.jpg')
os.remove('./train/dog.5604.jpg')
os.remove('./train/dog.1043.jpg')
os.remove('./train/cat.4338.jpg')
os.remove('./train/dog.10161.jpg')
os.remove('./train/dog.10190.jpg')

In [ ]:
tpath = "./train"
ftrain = get_image_files(tpath)
print('Train set size:', len(ftrain))

In [ ]:
# for debug shorter version
# ftrain = ftrain[:100]

In [ ]:
#import cv2
# test an image
#timg = cv2.imread('./train/dog.10190.jpg')
#plt.imshow(timg)

In [ ]:
import albumentations as Alb
class AlbTransform(Transform):
    def __init__(self, aug): self.aug = aug
    def encodes(self, img: PILImage):
        aug_img = self.aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_augs(): return Alb.Compose([
    Alb.ShiftScaleRotate(rotate_limit=20, border_mode=0, value=(0,0,0) ),
    Alb.Transpose(),
    Alb.Flip(),
    Alb.RandomRotate90(),
    Alb.RandomBrightnessContrast(),
    Alb.HueSaturationValue(
      hue_shift_limit=5, 
      sat_shift_limit=5, 
      val_shift_limit=5 ),    
])

item_tfms = [Resize(224), AlbTransform(get_augs())] 
batch_tfms = Normalize.from_stats(*imagenet_stats) 

In [ ]:
dls = ImageDataLoaders.from_name_re( path=tpath, 
    fnames=ftrain, pat=r'(.+)\.\d+.jpg$', valid_pct=0.1, 
    item_tfms=item_tfms, batch_tfms=batch_tfms, bs=64, shuffle=True )

In [ ]:
# uncomment to test data loaders
# dls.train.show_batch(max_n=12)
print('train items:', len(dls.train.items), 'validation items:', len(dls.valid.items))
# dls.valid.show_batch(max_n=12)
# dls.vocab

In [ ]:
learn = vision_learner(dls, densenet121, path='.', 
    loss_func=FocalLoss(),  
    metrics=[accuracy]  )

In [ ]:
learn.fine_tune(12, freeze_epochs=1, cbs=[ShowGraphCallback()])
print('fine_tune done')

In [ ]:
# import time
# startTime = time.time()

# lrs = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
# print('lr_find in:', time.time()-startTime, 'secs')

In [ ]:
# learn.fit_one_cycle(8, lr_max=lrs.valley, wd=0.2, cbs=[ShowGraphCallback()])
# print('training done')

In [ ]:
# prepare test data
ftest = get_image_files('test')
# ftest = ftest[:50] # scale down for debug
print('Testing', len(ftest), 'items')

In [ ]:
# make dataloader for test data
tst_dl = dls.test_dl(ftest, with_labels=False, shuffle=False)
# uncomment to see if dataloader is working
tst_dl.show_batch(max_n=12)

In [ ]:
startTime = time.time()
preds = learn.tta(dl=tst_dl, n=32, use_max=False)
print('TTA in:', time.time()-startTime, 'secs')

In [ ]:
# for this competition should not submit 0/1, should submit probability of "dog"
subm_df = pd.DataFrame()
subm_df['id'] = [item.stem for item in tst_dl.items]
subm_df['label'] = preds[0][:,1].clip(0.005, 0.995)
subm_df.to_csv('submission.csv', header=True, index=False)
subm_df

In [ ]:
subm_df['label'] = torch.softmax(preds[0], dim=1)[:, 1]
subm_df.to_csv('submission-softmax.csv', header=True, index=False)
subm_df

In [ ]:
# cleanup data
from shutil import rmtree
rmtree('./train', ignore_errors=True)
rmtree('./test', ignore_errors=True)

# Have a nice day!